<a href="https://colab.research.google.com/github/xuanyshi/clinical-trials-finer-grained-representation/blob/main/arm_level_rct_for_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import

In [ ]:
!pip install ipython-autotime
!pip install joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0:00:00


In [ ]:
import codecs
import xml.etree.ElementTree as ET
import sys
import csv
import pandas as pd
import numpy as np
import os
import json
import ast
import difflib
import zipfile
import ast
from collections import Counter
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from ast import literal_eval
import re
import scipy.stats as stats
from tqdm import tqdm
from datetime import datetime
import pytz
import torch
import numpy as np
import joblib
from joblib import Parallel, delayed
from scipy.spatial.distance import cdist

from sklearn.metrics.pairwise import cosine_similarity
import torch

%load_ext autotime

time: 344 µs (started: 2023-09-30 01:13:48 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 32.6 s (started: 2023-09-30 01:13:48 +00:00)


In [ ]:
tqdm.pandas()
beijing_tz = pytz.timezone('Asia/Shanghai')  # Beijing is in the 'Asia/Shanghai' timezone
beijing_time = datetime.now(beijing_tz)

time: 25.9 ms (started: 2023-09-07 00:10:00 +00:00)


# Read data

In [ ]:
df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/mti_results/mesh_outcome_df_0628.pickle')
df = df.rename(columns={'comparator_type':'comparator','comparator':'comparator_type'})

df['intervention'] =df['intervention'].apply(lambda x:ast.literal_eval(x))
df['intervention_type'] =df['intervention_type'].apply(lambda x:ast.literal_eval(x))
df['comparator'] =df['comparator'].apply(lambda x:ast.literal_eval(x))
df['comparator_type'] =df['comparator_type'].apply(lambda x:ast.literal_eval(x))
df['groups'] =df['groups'].apply(lambda x:ast.literal_eval(x))
df['intervention_mesh'] =df['intervention_mesh'].apply(lambda x:ast.literal_eval(x))

df['intervention_dict'] = df.apply(lambda row: [{k: v} for k, v in zip(row['intervention'], row['intervention_type'])], axis=1)
df['comparator_dict'] = df.apply(lambda row: [{k: v} for k, v in zip(row['comparator'], row['comparator_type'])], axis=1)

print(df.shape)
print(df['NCT_id'].nunique())

(222361, 42)
18720
time: 14.7 s (started: 2023-08-09 02:06:57 +00:00)


In [ ]:
df = df[df['groups'].str.len()==2]
print(df.shape)
print(df['NCT_id'].nunique())

(191441, 42)
15742
time: 295 ms (started: 2023-08-09 02:07:12 +00:00)


In [ ]:
df[['intervention','intervention_dict','comparator','comparator_dict','groups']].sample(5)

,intervention,intervention_dict,comparator,comparator_dict,groups
155069,"[Danirixin, Placebo]","[{'Danirixin': 'Drug'}, {'Placebo': 'Drug'}]",[],[],"[Placebo, DNX 75 mg]"
182862,"[Fezolinetant, Placebo]","[{'Fezolinetant': 'Drug'}, {'Placebo': 'Drug'}]",[],[],"[Placebo, Fezolinetant 15 mg BID]"
37601,"[Relugolix, Estradiol/norethindrone acetate]","[{'Relugolix': 'Drug'}, {'Estradiol/norethindr...","[Relugolix placebo, Estradiol/norethindrone ac...","[{'Relugolix placebo': 'Drug'}, {'Estradiol/no...","[Relugolix Plus E2/NETA (Group A), Placebo (Gr..."
157682,[Iota-Carrageenan],[{'Iota-Carrageenan': 'Device'}],[Placebo],[{'Placebo': 'Device'}],"[Placebo, Bisolviral]"
125031,[Novel Ibuprofen],[{'Novel Ibuprofen': 'Drug'}],"[Standard Ibuprofen, Placebo]","[{'Standard Ibuprofen': 'Drug'}, {'Placebo': '...","[Ibuprofen Sodium, Ibuprofen (Motrin IB)]"


time: 78.9 ms (started: 2023-08-09 02:07:12 +00:00)


In [ ]:
print(df.columns)
print(df.shape)

In [ ]:
df_new = df[df.apply(lambda row: all(len(row[col]) > 0 for col in ['intervention', 'comparator', 'groups']), axis=1)]
print(df_new.shape)
print(df_new['NCT_id'].nunique())

(119968, 42)
8665
time: 1.67 s (started: 2023-08-09 02:07:16 +00:00)


In [ ]:
full_df = df_new
sample_df = df_new.sample(50)
#[['NCT_id','intervention_mesh','all_inters','intervention','comparator','groups','outcome_title','label']].to_excel(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/mti_results/df_new_arm.xlsx')

time: 5.44 ms (started: 2023-08-08 00:52:31 +00:00)


# biobert classification (inter vs compa)

In [ ]:
!pip install transformers

time: 4.02 s (started: 2023-08-10 01:54:19 +00:00)


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load the BioBERT model and tokenizer
model_name = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to('cuda')

time: 12.3 s (started: 2023-08-10 03:20:11 +00:00)


In [ ]:
torch.cuda.is_available()

True

time: 2.81 ms (started: 2023-08-10 02:24:13 +00:00)


In [ ]:
# Function to generate sentence embedding
def get_sentence_embedding(sentence):
    device = 'cuda'
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    outputs = model(**inputs)
    sentence_embedding = torch.mean(outputs.last_hidden_state[0], dim=0)
    return sentence_embedding

# Generate embeddings for two sentences
sentence1 = "covid-19"
sentence2 = "coronavirus"
embedding1 = get_sentence_embedding(sentence1)
embedding2 = get_sentence_embedding(sentence2)

# Compute the cosine similarity of the two embeddings
similarity = 1 - cosine(embedding1.cpu().detach().numpy(), embedding2.cpu().detach().numpy())
print(f"The similarity between the two sentences is: {similarity}")

The similarity between the two sentences is: 0.8009061217308044
time: 515 ms (started: 2023-08-08 00:56:53 +00:00)


In [ ]:
#test
from scipy.spatial.distance import cdist

def get_batch_embeddings(sentences):
    device = 'cuda'  # define your device here
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).cpu().detach().numpy()
    return embeddings

def label_groups_with_embeddings_dict(row):
    groups = row["groups"]
    intervention_dicts = row["intervention_dict"]
    comparator_dicts = row["comparator_dict"]

    group_embeddings = get_batch_embeddings(groups)

    intervention_group = None
    intervention_value = None
    comparator_group = None
    comparator_value = None

    max_intervention_similarity = 0
    max_comparator_similarity = 0

    for group, group_embedding in zip(groups, group_embeddings):
        for intervention_dict in intervention_dicts:
            for key, value in intervention_dict.items():
                key_embedding = get_batch_embeddings([key])
                intervention_similarity = 1 - cdist(group_embedding[np.newaxis], key_embedding, 'cosine')[0][0]

                if intervention_similarity > max_intervention_similarity:
                    max_intervention_similarity = intervention_similarity
                    intervention_group = group
                    intervention_value = value

        for comparator_dict in comparator_dicts:
            for key, value in comparator_dict.items():
                key_embedding = get_batch_embeddings([key])
                comparator_similarity = 1 - cdist(group_embedding[np.newaxis], key_embedding, 'cosine')[0][0]

                if comparator_similarity > max_comparator_similarity:
                    max_comparator_similarity = comparator_similarity
                    comparator_group = group
                    comparator_value = value

    return pd.Series([
        intervention_group, intervention_value, max_intervention_similarity,
        comparator_group, comparator_value, max_comparator_similarity
    ])

time: 1.04 ms (started: 2023-08-08 00:56:57 +00:00)


In [ ]:
# from scipy.spatial.distance import cdist

# def get_batch_embeddings(sentences):
#     device = 'cuda'  # define your device here
#     inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(device)
#     outputs = model(**inputs)
#     embeddings = torch.mean(outputs.last_hidden_state, dim=1).cpu().detach().numpy()
#     return embeddings

# def label_groups_with_embeddings_dict(row):
#     groups = row["groups"]
#     intervention_dicts = row["intervention_dict"]  # If this is already a list of dictionaries
#     comparator_dicts = row["comparator_dict"]  # Make sure this column exists

#     group_embeddings = get_batch_embeddings(groups)

#     intervention_group = None
#     intervention_value = None
#     comparator_group = None
#     comparator_value = None

#     max_intervention_similarity = 0
#     max_comparator_similarity = 0

#     for group, group_embedding in zip(groups, group_embeddings):
#         for intervention_dict in intervention_dicts:
#             for key, value in intervention_dict.items():
#                 key_embedding = get_batch_embeddings([key])
#                 intervention_similarity = 1 - cdist(group_embedding[np.newaxis], key_embedding, 'cosine')

#                 if intervention_similarity >= 0.8 and intervention_similarity > max_intervention_similarity:
#                     max_intervention_similarity = intervention_similarity
#                     intervention_group = group
#                     intervention_value = value  # store the value, not the key

#         for comparator_dict in comparator_dicts:
#             for key, value in comparator_dict.items():
#                 key_embedding = get_batch_embeddings([key])
#                 comparator_similarity = 1 - cdist(group_embedding[np.newaxis], key_embedding, 'cosine')

#                 if comparator_similarity >= 0.8 and comparator_similarity > max_comparator_similarity:
#                     max_comparator_similarity = comparator_similarity
#                     comparator_group = group
#                     comparator_value = value  # store the value, not the key

#     return pd.Series([intervention_group, intervention_value, comparator_group, comparator_value])

time: 1.93 ms (started: 2023-07-27 14:10:03 +00:00)


### test

In [ ]:
import subprocess as sp
import os

def get_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

get_gpu_memory()

[14156]

time: 36.8 ms (started: 2023-08-09 13:20:19 +00:00)


In [ ]:
!nvidia-smi

Wed Aug  9 13:19:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    30W /  70W |    945MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
sample_sample = df_new.sample(200,random_state=2023)
sample_sample.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/sample_sample.csv')

time: 820 ms (started: 2023-08-08 00:57:12 +00:00)


In [ ]:
print(sample_sample.shape)

new_columns = ["intervention_group", "intervention_value", "intervention_similarity",
               "comparator_group", "comparator_value", "comparator_similarity"]

sample_sample[new_columns] = sample_sample.progress_apply(label_groups_with_embeddings_dict, axis=1)

sample_sample.dropna(subset=['intervention_group', 'comparator_group'], inplace=True)
print(sample_sample.shape)
sample_sample.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_sample.csv')

sample_sample[["groups","intervention_group", "intervention_value", "comparator_group", "comparator_value"]].sample(5)

(200, 42)


100%|██████████| 200/200 [00:21<00:00,  9.18it/s]


(200, 48)


,groups,intervention_group,intervention_value,comparator_group,comparator_value
166067,"[QVM149 150/50/80 µg o.d., QMF149 150/160 µg o...",QVM149 150/50/80 µg o.d.,Drug,QMF149 150/160 µg o.d.,Drug
59512,"[Placebo, ZOMIG 5 mg]",ZOMIG 5 mg,Drug,Placebo,Drug
181206,"[Placebo, Nemolizumab (90 mg)]",Nemolizumab (90 mg),Drug,Placebo,Drug
86445,"[AZD7442, Placebo]",AZD7442,Drug,Placebo,Drug
73706,"[PRM-151 10 mg/kg, Placebo]",PRM-151 10 mg/kg,Biological,Placebo,Other


time: 22.4 s (started: 2023-08-08 00:57:19 +00:00)


### not test

In [ ]:
beijing_time = datetime.now(beijing_tz)
print(beijing_time)

print(full_df.shape)

new_columns = ["intervention_group", "intervention_value", "intervention_similarity",
               "comparator_group", "comparator_value", "comparator_similarity"]

full_df[new_columns] = full_df.progress_apply(label_groups_with_embeddings_dict, axis=1)

#full_df[["intervention_group", "intervention_value", "comparator_group", "comparator_value"]] = full_df.progress_apply(label_groups_with_embeddings_dict, axis=1)
full_df.dropna(subset=['intervention_group', 'comparator_group'], inplace=True)

print(full_df.shape)

full_df.to_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
full_df.to_json(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.json')
full_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.csv')

2023-08-07 22:49:01.585081+08:00
(119968, 48)


100%|██████████| 119968/119968 [3:40:29<00:00,  9.07it/s]
<ipython-input-32-5f5e20633b82>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df[new_columns] = full_df.progress_apply(label_groups_with_embeddings_dict, axis=1)
<ipython-input-32-5f5e20633b82>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_df.dropna(subset=['intervention_group', 'comparator_group'], inplace=True)


(119968, 48)
time: 3h 40min 38s (started: 2023-08-07 14:49:01 +00:00)


#KG

In [ ]:
# kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df.pickle')
# print(kg_df.shape)
# print(kg_df['NCT_id'].nunique())

kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)
print(kg_df.shape)
print(kg_df['NCT_ID'].nunique())

(119968, 48)
8665
time: 1.59 s (started: 2023-08-11 03:19:50 +00:00)


In [ ]:
kg_df_sample = kg_df.sample(20,random_state=2023)
kg_df_sample = kg_df[kg_df['NCT_ID'].isin(kg_df_sample['NCT_ID'].unique())]
print(kg_df_sample.shape)
kg_df_sample.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/kg_df_sample.csv')

kg_df.sample(5,random_state=2023)[["groups","intervention",'comparator',"intervention_group", "intervention_value", "intervention_similarity",
               "comparator_group", "comparator_value", "comparator_similarity"]]

(1541, 48)


,groups,intervention,comparator,intervention_group,intervention_value,intervention_similarity,comparator_group,comparator_value,comparator_similarity
159708,"[Placebo, GSK2269557 1000 µg]",[GSK2269557],[Placebo],GSK2269557 1000 µg,Drug,0.921315,Placebo,Drug,0.886400
41891,"[PF-04965842 200 mg, Placebo]","[PF-04965842 100 mg, PF-04965842 200 mg]",[Placebo],PF-04965842 200 mg,Drug,1.000000,Placebo,Drug,0.894706
26247,"[Placebo, Bococizumab (PF-04950615)]",[bococizumab (PF-04950615)],[Placebo],Bococizumab (PF-04950615),Drug,0.963027,Placebo,Drug,0.864217
126213,[Integrated Study (Main + Japan Sub-study): Pl...,"[Adalimumab, Prednisone]",[Placebo],Integrated Study (Main + Japan Sub-study): Ada...,Drug,0.826469,Integrated Study (Main + Japan Sub-study): Pla...,Drug,0.745709
208367,"[PF-04937319 Once-Daily (300 mg), Sitagliptin ...","[PF-04937319 once-daily, PF-04937319 split-dose]",[Sitagliptin once-daily],PF-04937319 Once-Daily (300 mg),Drug,0.961127,Sitagliptin 100 mg,Drug,0.919221


time: 399 ms (started: 2023-08-11 03:19:57 +00:00)


# ADE

In [ ]:
files = os.listdir(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/og_files')

time: 7.7 ms (started: 2023-08-11 02:19:18 +00:00)


In [ ]:
file_list = []
for file in files:
  df_ade = pd.read_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/og_files/{file}')
  print(df_ade.shape)
  file_list.append(df_ade)

df_ade = pd.concat(file_list).drop(columns=['Unnamed: 0'])
print(df_ade.shape)
print(df_ade['NCT_ID'].nunique())

(305686, 11)
(269295, 11)
(623090, 11)
(612090, 11)
(1810161, 10)
18508
time: 5.03 s (started: 2023-08-11 02:19:18 +00:00)


In [ ]:
df_ade_sample = df_ade[df_ade['NCT_ID'].isin(kg_df_sample['NCT_id'].unique())]
print(df_ade_sample.shape)
df_ade_sample.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/df_ade_sample.csv')

(3067, 10)
time: 444 ms (started: 2023-08-11 02:19:23 +00:00)


In [ ]:
df_ade.sample(5)

,NCT_ID,arm_title,sub_title,category,serious/other,affected,at_risk,events,ratio,edge_type
168078,NCT03434379,Atezolizumab + Bevacizumab - China,Varices oesophageal,Gastrointestinal disorders,serious,0.0,132.0,0.0,0/.132,
181006,NCT00598273,Peginesatide 0.025 mg/kg,Cerebrovascular accident,Nervous system disorders,serious,0.0,161.0,0.0,0/.161,
64033,NCT02229851,Somapacitan - Main Phase,Cholangiogram,Investigations,serious,0.0,120.0,0.0,0/.120,
50674,NCT02503202,V920 Consistency Lot C,Back pain,Musculoskeletal and connective tissue disorders,serious,0.0,263.0,0.0,0/.263,
441506,NCT01599650,Ranibizumab 0.5mg + Laser,Cardiac failure,Cardiac disorders,serious,1.0,183.0,1.0,1/.183,has_ae


time: 104 ms (started: 2023-08-11 02:19:24 +00:00)


# Matching ADE and KG

## method 1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm


# Rename the 'NCT_id' column in kg_df_sample to match with 'NCT_ID' in df_ade_sample
kg_df_sample = kg_df_sample.copy(deep=True)
kg_df_sample.rename(columns={'NCT_id': 'NCT_ID'}, inplace=True)

# only keep ADE with valid affected number
df_ade_sample = df_ade_sample[df_ade_sample['affected']!=0]

# Merge the dataframes based on the matching criteria
matched_rows = pd.merge(df_ade_sample, kg_df_sample,
                        how='left',
                        left_on=['NCT_ID', 'arm_title'],
                        right_on=['NCT_ID', 'intervention_group'])

# Identify unmatched arm_titles (those with NaN in intervention_group column)
unmatched_rows = matched_rows[matched_rows['intervention_group'].isna()]

# Function to calculate embeddings
def calculate_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to GPU if available
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move the result back to CPU


# Create a list to store the result rows
result_rows = []

# Loop through each unique NCT_ID
for nct_id in tqdm(df_ade_sample['NCT_ID'].unique()):
    study_rows = df_ade_sample[df_ade_sample['NCT_ID'] == nct_id]
    kg_study_rows = kg_df_sample[kg_df_sample['NCT_ID'] == nct_id]

    # Get unique intervention and comparator groups
    unique_intervention_groups = kg_study_rows['intervention_group'].unique()
    unique_comparator_groups = kg_study_rows['comparator_group'].unique()

    # Loop through each arm_title within the study
    for _, row in study_rows.iterrows():
        arm_title = row['arm_title']
        matched_group = None
        similarity = None

        # Step 1: Match by intervention_group
        if arm_title in unique_intervention_groups:
            matched_group = arm_title

        # Step 2: Match by comparator_group (if no match in step 1)
        elif arm_title in unique_comparator_groups:
            matched_group = arm_title

        # Step 3: Apply BioBERT (if no match in steps 1 and 2)
        else:
            arm_title_embedding = calculate_embedding(arm_title)
            max_similarity = -1
            for group_type, unique_groups in [('intervention_group', unique_intervention_groups), ('comparator_group', unique_comparator_groups)]:
                for group in unique_groups:
                    group_embedding = calculate_embedding(group)
                    current_similarity = cosine_similarity(arm_title_embedding.reshape(1, -1), group_embedding.reshape(1, -1))[0][0]
                    if current_similarity > max_similarity:
                        max_similarity = current_similarity
                        matched_group = group
                        similarity = max_similarity

        # Append the result as a dictionary to the result_rows list
        result_rows.append({
            'NCT_ID': nct_id,
            'arm_title': arm_title,
            'matched_group': matched_group,
            'similarity': similarity
        })

# Convert the result_rows list to a DataFrame
final_results = pd.DataFrame(result_rows)

# Merge with the original DataFrame (if needed)
final_df = pd.merge(df_ade_sample, final_results, how='left', on=['NCT_ID', 'arm_title'])

# Replace 'path/to/df_ade_sample.csv' and 'path/to/kg_df_sample.csv' with the actual paths to your files
# Replace 'biobert-path' with the correct path to the BioBERT model

100%|██████████| 14/14 [00:13<00:00,  1.04it/s]

time: 13.6 s (started: 2023-08-09 13:26:44 +00:00)


In [ ]:
test_df = final_df[['NCT_ID','arm_title','sub_title','affected','at_risk','ratio','matched_group','similarity']].drop_duplicates(subset=['NCT_ID','arm_title','sub_title'])
test_df = test_df[test_df['affected']!=0]
print(test_df.shape)
test_df.sample(10,random_state=20)

(1469, 8)


,NCT_ID,arm_title,sub_title,affected,at_risk,ratio,matched_group,similarity
224793,NCT01975376,Placebo,Confusional state,2.0,8374.0,2/.8374,Placebo,NaN
47118,NCT01975376,Placebo,Constipation,1.0,8374.0,1/.8374,Placebo,NaN
10281,NCT01212770,APR Exposure Period Up to 5 Years: Apremilast ...,Cholecystitis,1.0,242.0,1/.242,Apremilast 30 mg,0.841438
280892,NCT01975376,Bococizumab (PF-04950615),Haemorrhagic anaemia,2.0,8386.0,2/.8386,Bococizumab (PF-04950615),NaN
53298,NCT01975376,Placebo,Haematemesis,1.0,8374.0,1/.8374,Placebo,NaN
159903,NCT01975376,Placebo,Bursitis,1.0,8374.0,1/.8374,Placebo,NaN
532783,NCT03070782,Placebo,Gastrointestinal anastomotic stenosis,1.0,47.0,1/.47,Placebo,NaN
3804,NCT01236547,"(Phase II) Pazopanib, Paclitaxel, IMRT",Neck pain,1.0,36.0,1/.36,"(Phase II) Pazopanib, Paclitaxel, IMRT",NaN
532445,NCT03084796,Treatment E,"Total, serious adverse events",1.0,121.0,1/.121,Treatment E,NaN
139303,NCT01975376,Placebo,Subdural haematoma,4.0,8374.0,4/.8374,Placebo,NaN


time: 154 ms (started: 2023-08-09 13:27:07 +00:00)


## method 2 batch process

In [ ]:
kg_df_sample = kg_df.copy(deep=True)
df_ade_sample = df_ade[df_ade['affected']!=0].copy(deep=True)
kg_df_sample.rename(columns={'NCT_id': 'NCT_ID'}, inplace=True)

time: 247 ms (started: 2023-08-11 03:20:08 +00:00)


In [ ]:
print(kg_df_sample.shape,df_ade_sample.shape,df_ade.shape)

(119968, 48) (804812, 10) (1810161, 10)
time: 561 µs (started: 2023-08-11 03:20:12 +00:00)


In [ ]:
print(df_ade['NCT_ID'].nunique(),df_ade_sample['NCT_ID'].nunique())

18508 11729
time: 152 ms (started: 2023-08-11 03:20:14 +00:00)


In [ ]:
# Function to calculate embeddings for a batch of texts
def calculate_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to GPU if available
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # Move the result back to CPU

# Create a list to store the result rows
result_rows = []

# Loop through each unique NCT_ID
for nct_id in tqdm(df_ade_sample['NCT_ID'].unique()):
    study_rows = df_ade_sample[df_ade_sample['NCT_ID'] == nct_id]
    kg_study_rows = kg_df_sample[kg_df_sample['NCT_ID'] == nct_id]

    # Get unique intervention and comparator groups
    unique_groups = np.concatenate([kg_study_rows['intervention_group'].dropna().unique(),
                                    kg_study_rows['comparator_group'].dropna().unique()])
    unique_groups = [str(group) for group in unique_groups if str(group).strip()]  # Convert to a list of non-empty strings

    # Check if unique_groups is not empty
    if unique_groups:
        # Calculate embeddings for unique groups only once
        unique_group_embeddings = calculate_embeddings(unique_groups)
    else:
        continue  # Skip this NCT_ID if there are no valid unique groups

    # Dictionary to store matched groups for identical arm_titles
    arm_title_matched_groups = {}

    # Process arm titles in batches
    batch_size = 15  # Adjust based on available GPU memory
    for i in range(0, len(study_rows), batch_size):
        batch_arm_titles = study_rows['arm_title'].iloc[i:i+batch_size].tolist()

        # Calculate embeddings for the batch of arm titles
        arm_title_embeddings = calculate_embeddings(batch_arm_titles)

        # Process each arm title in the batch
        for arm_title, arm_title_embedding in zip(batch_arm_titles, arm_title_embeddings):
            # Check if this arm_title has already been matched
            if arm_title in arm_title_matched_groups:
                matched_group, similarity = arm_title_matched_groups[arm_title]
            else:
                matched_group = None
                similarity = None

                # Match by unique_groups
                if arm_title in unique_groups:
                    matched_group = arm_title
                else:
                    # Apply BioBERT
                    similarities = cosine_similarity(arm_title_embedding.reshape(1, -1), unique_group_embeddings)
                    max_similarity_idx = np.argmax(similarities)
                    matched_group = unique_groups[max_similarity_idx]
                    similarity = similarities[0, max_similarity_idx]

                # Store the matched group for identical arm_titles
                arm_title_matched_groups[arm_title] = (matched_group, similarity)

            # Append the result as a dictionary to the result_rows list
            result_rows.append({
                'NCT_ID': nct_id,
                'arm_title': arm_title,
                'matched_group': matched_group,
                'similarity': similarity
            })

# Convert the result_rows list to a DataFrame
final_results = pd.DataFrame(result_rows)
print(final_results.shape,df_ade_sample.shape)

100%|██████████| 11729/11729 [21:50<00:00,  8.95it/s]


(538311, 4) (804812, 10)
time: 21min 50s (started: 2023-08-10 03:34:19 +00:00)


In [ ]:
final_results.to_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/final_results.pickle')

time: 108 ms (started: 2023-08-10 03:57:58 +00:00)


In [ ]:
final_results = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/final_results.pickle')

time: 97.5 ms (started: 2023-08-11 03:20:20 +00:00)


In [ ]:
# Merge with the original DataFrame (if needed)

# chunk_size = 45000  # Adjust based on available memory
# final_df = pd.DataFrame()

# for i in tqdm(range(0, len(df_ade_sample), chunk_size)):
#     chunk = df_ade_sample.iloc[i:i + chunk_size]
#     merged_chunk = pd.merge(chunk, final_results, how='left', on=['NCT_ID', 'arm_title'])
#     final_df = pd.concat([final_df, merged_chunk])

# final_df = final_df.drop_duplicates()
# print(final_df.shape)
# Now final_df contains the merged data

final_df = pd.merge(df_ade_sample, final_results.drop_duplicates(), how='left', on=['NCT_ID', 'arm_title'])

# Replace 'path/to/df_ade_sample.csv' and 'path/to/kg_df_sample.csv' with the actual paths to your files
# Replace 'biobert-path' with the correct path to the BioBERT model

time: 469 ms (started: 2023-08-11 02:27:49 +00:00)


In [ ]:
ade_result_matched_df = final_df.drop_duplicates(subset=['NCT_ID','arm_title','sub_title'])
print(ade_result_matched_df.shape)
ade_result_matched_df['matched_group'].fillna(ade_result_matched_df['arm_title'],inplace=True)
ade_result_matched_df.to_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
ade_result_matched_df.to_json(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.json')
ade_result_matched_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.csv')

(803052, 12)


<ipython-input-65-2240fd3cea16>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ade_result_matched_df['matched_group'].fillna(ade_result_matched_df['arm_title'],inplace=True)


time: 13.8 s (started: 2023-08-11 07:52:06 +00:00)


In [ ]:
ade_result_matched_df = pd.read_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.csv')

time: 2.1 s (started: 2023-09-07 00:14:22 +00:00)


In [ ]:
sample = ade_result_matched_df.sample(5,random_state=95)[['NCT_ID','arm_title','sub_title','category','serious/other','affected','at_risk','matched_group']]
sample.to_excel(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ade_sample.xlsx')
sample

,NCT_ID,arm_title,sub_title,category,serious/other,affected,at_risk,matched_group
386146,NCT00977080,Oral Paricalcitol in the Oral Stratum,Abdominal pain,Gastrointestinal disorders,serious,1.0,72.0,Oral Paricalcitol in the Oral Stratum
544911,NCT02254278,IMRT 5 Weeks,Syncope,Nervous system disorders,serious,1.0,147.0,IMRT 5 Weeks
672578,NCT00509145,Laquinimod 0.6 mg,Uterine prolapse,Reproductive system and breast disorders,serious,1.0,550.0,Laquinimod
701901,NCT00810199,Tocilizumab + Placebo,Clostridium difficile colitis,Infections and infestations,serious,1.0,276.0,Tocilizumab + Placebo
580468,NCT02853331,Sunitinib,Hepatocellular injury,Hepatobiliary disorders,serious,1.0,425.0,Sunitinib


time: 77.5 ms (started: 2023-09-07 00:18:18 +00:00)


# Matching ARM with Intervention_MeSH

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)
print(kg_df.shape)
print(kg_df['NCT_ID'].nunique())

(119968, 48)
8665
time: 5.41 s (started: 2023-09-16 12:35:11 +00:00)


In [ ]:
kg_df[['outcome_type']].value_counts()

outcome_type       
Secondary              92966
Primary                23252
Other Pre-specified     3452
Post-Hoc                 298
dtype: int64

time: 22.2 ms (started: 2023-09-16 12:35:39 +00:00)


In [ ]:
inter_group_mesh_rel = kg_df[kg_df['intervention_mesh'].str.len()==1][['intervention_group','intervention_mesh']]
print(inter_group_mesh_rel.shape)
inter_group_mesh_rel['intervention_mesh'] = inter_group_mesh_rel['intervention_mesh'].apply(lambda x:x[0])
inter_group_mesh_rel.drop_duplicates()
print(inter_group_mesh_rel.shape)
inter_group_mesh_rel['relation'] = 'is_a'

inter_group_mesh_rel.to_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')
inter_group_mesh_rel[['intervention_group','intervention_mesh','relation']].sample(5)

(49764, 2)
(49764, 2)


,intervention_group,intervention_mesh,relation
12309,Duloxetine,Duloxetine Hydrochloride,is_a
144164,Brexpiprazole,Brexpiprazole,is_a
101254,Preladenant 1 mg BID,Levodopa,is_a
14606,Ibuprofen Sodium,Ibuprofen,is_a
8835,Tanezumab 5 mg,Tanezumab,is_a


time: 234 ms (started: 2023-08-29 08:10:26 +00:00)


# case : infliximab vs infections

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)

ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

time: 5.73 s (started: 2023-09-15 08:11:40 +00:00)


## check how many trials have both ae and efficacy

In [ ]:
print(ade_result_matched_df.shape,ade_result_matched_df['NCT_ID'].nunique())

(803052, 12) 11729
time: 54.4 ms (started: 2023-09-15 08:24:50 +00:00)


In [ ]:
print(kg_df.shape,kg_df['NCT_ID'].nunique())

(119968, 48) 8665
time: 16 ms (started: 2023-09-15 08:45:07 +00:00)


In [ ]:
shared = list(set(ade_result_matched_df['NCT_ID'].unique()).intersection(set(kg_df['NCT_ID'].unique())))
print(len(shared))

6429
time: 60 ms (started: 2023-09-15 08:50:26 +00:00)


In [ ]:
shared[:10]

['NCT00838916',
 'NCT00450216',
 'NCT01154088',
 'NCT03082196',
 'NCT02413372',
 'NCT00681564',
 'NCT02117414',
 'NCT02265744',
 'NCT03036150',
 'NCT02139644']

time: 3.23 ms (started: 2023-09-15 08:52:36 +00:00)


In [ ]:
print(ade_result_matched_df['category'].isnull().sum())

0
time: 43.7 ms (started: 2023-09-07 02:56:39 +00:00)


In [ ]:
print(kg_df['intervention_group'].nunique())
print(kg_df['outcome_title'].nunique())
print(ade_result_matched_df['sub_title'].nunique())
print(ade_result_matched_df['category'].nunique())

print(kg_df.shape,ade_result_matched_df.shape)

10364
41708
40408
28
(119968, 48) (803052, 12)
time: 213 ms (started: 2023-09-07 02:47:55 +00:00)


In [ ]:
mesh_outcome_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/mti_results/mesh_outcome_df_0628.pickle')
print(mesh_outcome_df.shape,mesh_outcome_df.columns)

(222361, 40) Index(['NCT_id', 'study_type', 'completion_year', 'trial_phase',
       'enrollment_num', 'allocation', 'Countries', 'funder_name',
       'funder_type', 'condition_mesh', 'intervention_mesh', 'condition',
       'age', 'all_inters', 'intervention_type', 'intervention',
       'comparator_type', 'comparator', 'outcome_id', 'info', 'groups',
       'outcome_type', 'outcome_title', 'p_value', 'method', 'method_desc',
       'ci_percent', 'ci_n_sides', 'ci_lower_limit', 'ci_upper_limit',
       'param_type', 'param_value', 'split_p', 'p_split', 'label',
       'ci_lower_limit_clean', 'ci_upper_limit_clean', 'mesh_terms_mti',
       'mapped_mesh_terms', 'tree_level_0'],
      dtype='object')
time: 622 ms (started: 2023-09-07 02:59:19 +00:00)


## examples for group separation

In [ ]:
print(kg_df.columns)

Index(['NCT_ID', 'study_type', 'completion_year', 'trial_phase',
       'enrollment_num', 'allocation', 'Countries', 'funder_name',
       'funder_type', 'condition_mesh', 'intervention_mesh', 'condition',
       'age', 'all_inters', 'intervention_type', 'intervention', 'comparator',
       'comparator_type', 'outcome_id', 'info', 'groups', 'outcome_type',
       'outcome_title', 'p_value', 'method', 'method_desc', 'ci_percent',
       'ci_n_sides', 'ci_lower_limit', 'ci_upper_limit', 'param_type',
       'param_value', 'split_p', 'p_split', 'label', 'ci_lower_limit_clean',
       'ci_upper_limit_clean', 'mesh_terms_mti', 'mapped_mesh_terms',
       'tree_level_0', 'intervention_dict', 'comparator_dict',
       'intervention_group', 'intervention_value', 'intervention_similarity',
       'comparator_group', 'comparator_value', 'comparator_similarity'],
      dtype='object')
time: 872 µs (started: 2023-09-06 08:29:35 +00:00)


In [ ]:
kg_df.sample(10,random_state=230)[['NCT_ID','groups','intervention','comparator','intervention_group','comparator_group']]

,NCT_ID,groups,intervention,comparator,intervention_group,comparator_group
150032,NCT01694849,"[GFT505 80mg, Placebo]","[GFT505 80mg, GFT505 120mg]",[Placebo],GFT505 80mg,Placebo
206348,NCT01529346,"[Ibuprofen, Placebo]","[PF-05089771, Placebo, PF-05089771, Placebo, P...","[Ibuprofen, Placebo, Placebo, Placebo]",Placebo,Ibuprofen
130501,NCT01227564,"[ACC 3 μg+QS-21, Placebo]","[ACC-001 3 μg/ QS-21 50 μg, ACC-001 10 μg/ QS-...",[Placebo- Phosphate buffered saline (PBS)],ACC 3 μg+QS-21,Placebo
48001,NCT00402246,"[Remote Arm, In-Office Arm]",[Remote Management],[In-Office Care],Remote Arm,In-Office Arm
30618,NCT02421510,"[Placebo, Sotagliflozin 200 mg]","[Sotagliflozin, Sotagliflozin]",[Placebo],Sotagliflozin 200 mg,Placebo
210754,NCT03077438,"[Group 1: MenACYW Conjugate Vaccine, Group 2: ...","[Meningococcal Polysaccharide (Serogroups A, C...","[Meningococcal (Groups A, C, Y and W 135) Olig...",Group 1: MenACYW Conjugate Vaccine,Group 1: MenACYW Conjugate Vaccine
59821,NCT01244815,"[Placebo, Asenapine 2.5 mg BID]","[asenapine, Rescue medication]",[Placebo to match asenapine],Placebo,Placebo
175769,NCT02940886,"[Iron Isomaltoside/Ferric Derisomaltose, Iron ...",[Iron isomaltoside/ferric derisomaltose],[Iron sucrose],Iron Isomaltoside/Ferric Derisomaltose,Iron Sucrose
131993,NCT01255592,"[AZD5069, Placebo]",[AZD5069],[Placebo],AZD5069,Placebo
76815,NCT02819518,"[Part 2: Pembrolizumab + Chemotherapy, Part 2:...","[Pembrolizumab, Nab-paclitaxel, Paclitaxel, Ge...",[Normale Saline Solution],Part 2: Pembrolizumab + Chemotherapy,Part 2: Placebo + Chemotherapy


time: 27.6 ms (started: 2023-09-05 08:49:35 +00:00)


In [ ]:
sample = kg_df.sample(10,random_state=220)[['NCT_ID','groups','outcome_title','p_value']]
sample.to_excel(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/sample_effectiveness.xlsx')
sample

,NCT_ID,groups,outcome_title,p_value
68525,NCT01995838,"[Lemborexant 2.5 mg, Placebo]",Mean Change From Baseline in Sleep Efficiency ...,0.038
91447,NCT00089674,"[Placebo, Denosumab 60 mg Q6M]",Number of Participants With Any Fracture Throu...,0.7961
98329,NCT00348140,"[Placebo, RSG XR 2mg]",Change From Baseline in Alzheimer's Carer's Qu...,0.284
134858,NCT01360632,"[Brexpiprazole (1mg) + ADT, Placebo + ADT]",Change From Baseline (End of Phase A [Week 8])...,0.0816
113774,NCT00710554,"[Sativex, Placebo]",Change From Baseline in Quality of Life EuroQo...,0.617
124241,NCT01049373,"[Lymphdiaral Basistropfen (HDC), Placebo Solut...",Change in FFbH-R Between Screening and 2 Weeks,0.0757
39612,NCT03400800,"[Inclisiran, Placebo]",Percentage Change in LDL-C From Baseline to Da...,<0.0001
35500,NCT02912650,"[Placebo, Ibuprofen 250 mg]",Pain Intensity Difference on 11-Point Numerica...,<0.001
177120,NCT02954354,"[Baloxavir, Oseltamivir]",Percentage of Participants Reporting Normal Te...,0.5908
147362,NCT01795547,"[Aripiprazole, Paliperidone]",Change From Baseline to Week 28 in the 'Interp...,0.07


time: 339 ms (started: 2023-09-06 08:35:44 +00:00)


## case

In [ ]:
infliximab_ade = ade_result_matched_df[ade_result_matched_df['matched_group'].str.lower().str.contains('infliximab')]
infliximab_kg = kg_df[kg_df['intervention_group'].str.lower().str.contains('infliximab')]
infliximab_inter_mesh_rel = inter_group_mesh_rel[inter_group_mesh_rel['intervention_group'].str.lower().str.contains('infliximab')]

print(infliximab_ade.shape,infliximab_ade['NCT_ID'].nunique())
print(infliximab_kg.shape,infliximab_kg['NCT_ID'].nunique())
print(infliximab_inter_mesh_rel.shape)


infliximab_kg.rename(columns={'label':'significance'},inplace=True)
infliximab_ade.rename(columns={'serious/other':'serious_or_other'},inplace=True)

infliximab_ade.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/infliximab_ade.csv')
infliximab_kg.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/infliximab_kg.csv')
infliximab_inter_mesh_rel.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/infliximab_inter_mesh_rel.csv')


(481, 12) 22
(72, 48) 10
(38, 3)
time: 694 ms (started: 2023-08-29 08:11:23 +00:00)


<ipython-input-33-80b25e628d18>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infliximab_kg.rename(columns={'label':'significance'},inplace=True)
<ipython-input-33-80b25e628d18>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infliximab_ade.rename(columns={'serious/other':'serious_or_other'},inplace=True)


In [ ]:
infliximab_inter_mesh_rel

,intervention_group,intervention_mesh,relation
15972,Infliximab 5 mg/kg,Infliximab,is_a
17583,Infliximab,Infliximab,is_a
17584,Infliximab,Infliximab,is_a
60868,Infliximab,Infliximab,is_a
60869,Infliximab,Infliximab,is_a
60870,Infliximab,Infliximab,is_a
60871,Infliximab,Infliximab,is_a
96397,"Experimental: Infliximab (Remicade, Revellex)",Infliximab,is_a
96398,"Experimental: Infliximab (Remicade, Revellex)",Infliximab,is_a
96399,"Experimental: Infliximab (Remicade, Revellex)",Infliximab,is_a


time: 12.1 ms (started: 2023-08-29 10:57:36 +00:00)


In [ ]:
kg_df[kg_df['intervention_group'].str.lower().str.contains('infliximab')][['intervention_group','intervention_mesh']].sample(20)

,intervention_group,intervention_mesh
210170,Infliximab / ABP 710,[Infliximab]
185477,PF-06687234 20 mg + Infliximab,[]
17584,Infliximab,[Infliximab]
210150,Infliximab / ABP 710,[Infliximab]
210198,Infliximab / ABP 710,[Infliximab]
86625,Infliximab + Naproxen,"[Naproxen, Infliximab]"
96403,"Experimental: Infliximab (Remicade, Revellex)",[Infliximab]
210200,Infliximab / ABP 710,[Infliximab]
2000,Infliximab,"[Methotrexate, Infliximab]"
210172,Infliximab / ABP 710,[Infliximab]


time: 104 ms (started: 2023-08-11 12:09:32 +00:00)


In [ ]:
infliximab_ade.groupby('sub_title')['serious/other'].value_counts()

sub_title             serious/other
ABDOMINAL PAIN        serious          1
ABORTION INDUCED      serious          1
ABORTION SPONTANEOUS  serious          1
ALVEOLITIS            serious          1
ANAEMIA               serious          1
                                      ..
VIRAL INFECTION       serious          1
Venous stenosis       serious          1
Vertigo               serious          1
Viral Infection       serious          2
Vomiting              serious          1
Name: serious/other, Length: 344, dtype: int64

time: 15.5 ms (started: 2023-08-11 06:40:29 +00:00)


In [ ]:
final_df.drop_duplicates().shape

(400, 12)

time: 7.52 ms (started: 2023-08-10 03:31:00 +00:00)


In [ ]:
final_df.tail(10)

,NCT_ID,arm_title,sub_title,category,serious/other,affected,at_risk,events,ratio,edge_type,matched_group,similarity
42,NCT00793624,Placebo,Dehydration,Metabolism and nutrition disorders,serious,1.0,225.0,1.0,1/.225,has_ae,Placebo,NaN
43,NCT00790335,A-Intervention,Cholangitis,Hepatobiliary disorders,serious,1.0,336.0,1.0,1/.336,has_ae,NaN,NaN
44,NCT00303459,Placebo,CHOLECYSTITIS ACUTE,Hepatobiliary disorders,serious,1.0,174.0,1.0,1/.174,has_ae,Placebo,NaN
45,NCT01994889,Tafamidis 80 mg,Sinus bradycardia,Cardiac disorders,serious,1.0,176.0,1.0,1/.176,has_ae,Tafamidis,0.928323
46,NCT01497366,PEG+RBV,Infection,Infections and infestations,serious,1.0,243.0,1.0,1/.243,has_ae,PEG+RBV,NaN
47,NCT00330759,Denosumab 120 mg Q4W,Diabetes mellitus,Metabolism and nutrition disorders,serious,2.0,878.0,2.0,2/.878,has_ae,Denosumab,0.923565
48,NCT01928394,PC Arm N,Alanine aminotransferase increased,Investigations,serious,1.0,18.0,1.0,1/.18,has_ae,NaN,NaN
49,NCT03366337,Bardoxolone Methyl - ADPKD,Histoplasmosis disseminated,Infections and infestations,serious,1.0,31.0,1.0,1/.31,has_ae,NaN,NaN
50,NCT02174627,Placebo,Sinus arrest,Cardiac disorders,serious,1.0,1377.0,1.0,1/.1377,has_ae,Placebo,NaN
52,NCT01960348,Patisiran (ALN-TTR02),Oedema peripheral,General disorders,serious,1.0,148.0,1.0,1/.148,has_ae,NaN,NaN


time: 13.2 ms (started: 2023-08-10 03:31:00 +00:00)


In [ ]:
df_ade_sample.tail(10)

,NCT_ID,arm_title,sub_title,category,serious/other,affected,at_risk,events,ratio,edge_type
227689,NCT00793624,Placebo,Dehydration,Metabolism and nutrition disorders,serious,1.0,225.0,1.0,1/.225,has_ae
228135,NCT00790335,A-Intervention,Cholangitis,Hepatobiliary disorders,serious,1.0,336.0,1.0,1/.336,has_ae
301194,NCT00303459,Placebo,CHOLECYSTITIS ACUTE,Hepatobiliary disorders,serious,1.0,174.0,1.0,1/.174,has_ae
570242,NCT01994889,Tafamidis 80 mg,Sinus bradycardia,Cardiac disorders,serious,1.0,176.0,1.0,1/.176,has_ae
204944,NCT01497366,PEG+RBV,Infection,Infections and infestations,serious,1.0,243.0,1.0,1/.243,has_ae
291112,NCT00330759,Denosumab 120 mg Q4W,Diabetes mellitus,Metabolism and nutrition disorders,serious,2.0,878.0,2.0,2/.878,has_ae
514118,NCT01928394,PC Arm N,Alanine aminotransferase increased,Investigations,serious,1.0,18.0,1.0,1/.18,has_ae
165553,NCT03366337,Bardoxolone Methyl - ADPKD,Histoplasmosis disseminated,Infections and infestations,serious,1.0,31.0,1.0,1/.31,has_ae
615240,NCT02174627,Placebo,Sinus arrest,Cardiac disorders,serious,1.0,1377.0,1.0,1/.1377,has_ae
551008,NCT01960348,Patisiran (ALN-TTR02),Oedema peripheral,General disorders,serious,1.0,148.0,1.0,1/.148,has_ae


time: 11.1 ms (started: 2023-08-10 03:31:00 +00:00)


In [ ]:
kg_df[kg_df['NCT_id']=='NCT03366337']

,NCT_id,study_type,completion_year,trial_phase,enrollment_num,allocation,Countries,funder_name,funder_type,condition_mesh,...,mapped_mesh_terms,tree_level_0,intervention_dict,comparator_dict,intervention_group,intervention_value,intervention_similarity,comparator_group,comparator_value,comparator_similarity


time: 22.5 ms (started: 2023-08-10 03:31:37 +00:00)


# case : Cyclosporine vs kidney 环孢素 vs 肾损伤

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)

ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

time: 9.79 s (started: 2023-09-17 02:20:27 +00:00)


In [ ]:
cyclosporine_ade = ade_result_matched_df[ade_result_matched_df['matched_group'].str.lower().str.contains('cyclosporine')]
cyclosporine_ade = cyclosporine_ade[cyclosporine_ade['sub_title'].str.lower().str.contains('kidney')|cyclosporine_ade['category'].str.lower().str.contains('renal')]
cyclosporine_kg = kg_df[kg_df['intervention_group'].str.lower().str.contains('cyclosporine')]
cyclosporine_inter_mesh_rel = inter_group_mesh_rel[inter_group_mesh_rel['intervention_group'].str.lower().str.contains('cyclosporine')]

print(cyclosporine_ade.shape,cyclosporine_ade['NCT_ID'].nunique())
print(cyclosporine_kg.shape,cyclosporine_kg['NCT_ID'].nunique())
print(cyclosporine_inter_mesh_rel.shape)


cyclosporine_kg.rename(columns={'label':'significance'},inplace=True)
cyclosporine_ade.rename(columns={'serious/other':'serious_or_other'},inplace=True)

cyclosporine_ade.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/cyclosporine_ade.csv')
cyclosporine_kg.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/cyclosporine_kg.csv')
cyclosporine_inter_mesh_rel.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/cyclosporine_inter_mesh_rel.csv')


(77, 12) 6
(428, 48) 3
(0, 3)


<ipython-input-22-7f32ce1556de>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyclosporine_kg.rename(columns={'label':'significance'},inplace=True)


time: 1.2 s (started: 2023-09-17 07:39:43 +00:00)


In [ ]:
cyclosporine_kg['NCT_ID'].nunique()

3

time: 4.21 ms (started: 2023-09-17 02:46:45 +00:00)


In [ ]:
cyclosporine_kg[['NCT_ID','intervention_group','comparator_group','intervention','comparator']].sample(10)

,NCT_ID,intervention_group,comparator_group,intervention,comparator
111049,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
111109,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
110935,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
110901,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
110979,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
111078,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
110840,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
110873,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
51668,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]
111053,NCT00658359,Cyclosporine,Cyclosporine,"[CP-690,550, CP-690,550]",[Cyclosporine]


time: 64.6 ms (started: 2023-09-17 08:14:22 +00:00)


# case:Imipenem

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)

ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

In [ ]:
imipenem_ade = ade_result_matched_df[ade_result_matched_df['matched_group'].str.lower().str.contains('imipenem')|ade_result_matched_df['matched_group'].str.lower().str.contains('cilastatin')]
imipenem_ade = imipenem_ade[imipenem_ade['sub_title'].str.lower().str.contains('infect')|cyclosporine_ade['category'].str.lower().str.contains('infect')]
imipenem_kg = kg_df[kg_df['intervention_group'].str.lower().str.contains('imipenem')|kg_df['intervention_group'].str.lower().str.contains('cilastatin')]
imipenem_kg = imipenem_kg[imipenem_kg['condition'].astype(str).str.lower().str.contains('infect')]
imipenem_inter_mesh_rel = inter_group_mesh_rel[inter_group_mesh_rel['intervention_group'].str.lower().str.contains('imipenem')|inter_group_mesh_rel['intervention_group'].str.lower().str.contains('cilastatin')]

print(imipenem_ade.shape,imipenem_ade['NCT_ID'].nunique())
print(imipenem_kg.shape,imipenem_kg['NCT_ID'].nunique())
print(imipenem_inter_mesh_rel.shape)


imipenem_kg.rename(columns={'label':'significance'},inplace=True)
imipenem_ade.rename(columns={'serious/other':'serious_or_other'},inplace=True)

imipenem_ade.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/imipenem_ade.csv')
imipenem_kg.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/imipenem_kg.csv')
imipenem_inter_mesh_rel.to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/imipenem_inter_mesh_rel.csv')


<ipython-input-42-dffddd8eac7b>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  imipenem_ade = imipenem_ade[imipenem_ade['sub_title'].str.lower().str.contains('infect')|cyclosporine_ade['category'].str.lower().str.contains('infect')]


(12, 12) 6
(27, 48) 3
(2, 3)
time: 1.25 s (started: 2023-09-17 08:52:40 +00:00)


In [ ]:
imipenem_inter_mesh_rel

,intervention_group,intervention_mesh,relation
186044,Prolonged Infusion. Continuous Piperacillin/Ta...,Meropenem,is_a
186045,Prolonged Infusion. Continuous Piperacillin/Ta...,Meropenem,is_a


time: 7.59 ms (started: 2023-09-17 08:34:49 +00:00)


In [ ]:
imipenem_kg[['NCT_ID','intervention_group','condition','significance']]

,NCT_ID,intervention_group,condition,significance
20301,NCT01505634,Relebactam 125 mg With Imipenem/Cilastatin,"['Urinary Tract Infections', 'Pyelonephritis']",positive
20323,NCT01506271,Relebactam 250 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20324,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20325,NCT01506271,Relebactam 250 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20326,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20327,NCT01506271,Relebactam 250 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20328,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20329,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20330,NCT01506271,Relebactam 250 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive
20331,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,['Intra-abdominal Infections'],positive


time: 11.5 ms (started: 2023-09-17 09:45:17 +00:00)


In [ ]:
imipenem_ade[['NCT_ID','arm_title','sub_title','category','serious_or_other','affected','at_risk']]

,NCT_ID,arm_title,sub_title,category,serious_or_other,affected,at_risk
77880,NCT00515034,cIAI Treated With Imipenem/Cilastatin,urinary tract infection,Infections and infestations,serious,1.0,19.0
100080,NCT02321800,Imipenem/Cilastatin,Device related infection,Infections and infestations,serious,1.0,148.0
204912,NCT00589693,Imipenem-cilastatin,Urinary tract infection,Infections and infestations,serious,1.0,112.0
204916,NCT00589693,Imipenem-cilastatin,Device related infection,Infections and infestations,serious,1.0,112.0
204917,NCT00589693,Imipenem-cilastatin,Fungal infection,Infections and infestations,serious,1.0,112.0
204918,NCT00589693,Imipenem-cilastatin,Lung infection,Infections and infestations,serious,1.0,112.0
204922,NCT00589693,Imipenem-cilastatin,Postoperative wound infection,Infections and infestations,serious,1.0,112.0
466740,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,Clostridium difficile infection,Infections and infestations,serious,1.0,116.0
466741,NCT01506271,Relebactam 125 mg With Imipenem/Cilastatin,Postoperative wound infection,Infections and infestations,serious,1.0,116.0
752079,NCT01721408,Imipenem/Cilastatin,Postoperative wound infection,Infections and infestations,serious,2.0,232.0


time: 15.7 ms (started: 2023-09-17 09:53:11 +00:00)


In [ ]:
imipenem_ade[['NCT_ID','arm_title','sub_title','category','serious_or_other','affected','at_risk']].to_csv(f'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/samples/imipenem_ade_for_manu.csv')

time: 11.9 ms (started: 2023-09-17 10:05:44 +00:00)


# case: infections

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)

ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

kg_df.rename(columns={'label':'significance'},inplace=True)
ade_result_matched_df.rename(columns={'serious/other':'serious_or_other'},inplace=True)

time: 6 s (started: 2023-09-27 00:20:45 +00:00)


In [ ]:
infection_kg_df = kg_df[kg_df['condition'].astype(str).str.lower().str.contains('infect')]
infection_kg_df['condition_mesh'] = infection_kg_df['condition_mesh'].apply(lambda x:ast.literal_eval(x))
infection_kg_df = infection_kg_df[infection_kg_df['condition_mesh'].str.len()==1]
infection_kg_df['condition_mesh'] = infection_kg_df['condition_mesh'].apply(lambda x:x[0])

infection_ade_df = ade_result_matched_df[ade_result_matched_df['sub_title'].str.lower().str.contains('infect')|ade_result_matched_df['category'].str.lower().str.contains('infect')]
infection_ade_df = infection_ade_df[infection_ade_df['matched_group'].isin(infection_kg_df['intervention_group'].unique())]

<ipython-input-5-cac94ad50a2a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infection_kg_df['condition_mesh'] = infection_kg_df['condition_mesh'].apply(lambda x:ast.literal_eval(x))


time: 1.46 s (started: 2023-09-27 00:20:51 +00:00)


In [ ]:
infection_ade_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/infection_ade_df.csv')
infection_kg_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/infection_kg_df.csv')
inter_group_mesh_rel.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/inter_group_mesh_rel_0829.csv')

time: 545 ms (started: 2023-09-18 03:09:57 +00:00)


In [ ]:
infection_ade_df['matched_group'].nunique()

269

time: 4.59 ms (started: 2023-09-18 05:43:10 +00:00)


In [ ]:
infection_ade_df['matched_group'].value_counts()

Placebo                      11662
Palivizumab                    214
Ctrl-6W-6M/043+053 Group       169
Vancomycin                     166
E/C/F/TAF                      144
                             ...  
Standard Course Treatment        1
Concomitant Group                1
HIV-uninfected RotaTeq           1
V920 High-dose Lot               1
2% Bisphosphocin Nu-3            1
Name: matched_group, Length: 269, dtype: int64

time: 7.32 ms (started: 2023-09-18 05:43:23 +00:00)


In [ ]:
infection_kg_df['condition_mesh'].value_counts()

HIV Infections                            659
Pneumococcal Infections                   657
Infections                                326
Meningococcal Infections                  168
Peripheral Nervous System Diseases        156
Respiratory Tract Infections              130
Streptococcal Infections                  119
Bacterial Infections                       81
Influenza, Human                           67
Communicable Diseases                      55
Cytomegalovirus Infections                 53
Urinary Tract Infections                   50
Hemorrhagic Fever, Ebola                   39
Rotavirus Infections                       31
Respiratory Syncytial Virus Infections     21
COVID-19                                   18
Zika Virus Infection                       18
Rabies                                     17
Glycogen Storage Disease Type II           16
Candidiasis                                15
Staphylococcal Infections                  12
Sexually Transmitted Diseases     

time: 6.53 ms (started: 2023-09-18 06:06:38 +00:00)


In [ ]:
infection_ade_df['sub_title'].value_counts()

Pneumonia                          803
Urinary tract infection            372
Cellulitis                         341
Sepsis                             335
Gastroenteritis                    332
                                  ... 
Vestibuar Neuronitis                 1
Pumonary tuberculosis                1
Any Infections and Infestations      1
Any Infections and infestations      1
Pneumonia acinetobacter              1
Name: sub_title, Length: 1732, dtype: int64

time: 8.92 ms (started: 2023-09-18 06:07:31 +00:00)


## by mesh C01

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0807_no_thre.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)
ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

kg_df.rename(columns={'label':'significance'},inplace=True)
ade_result_matched_df.rename(columns={'serious/other':'serious_or_other'},inplace=True)

time: 2.32 s (started: 2023-09-20 07:56:32 +00:00)


### mesh_2021

In [ ]:
mesh_2021 = pd.read_csv(r'/content/drive/MyDrive/PKU/杜老师/AIreport/0929/MeSH_2021.csv').dropna(subset=['TREENUMBER'])
print(mesh_2021.shape)
print(mesh_2021.isnull().sum())
mesh_2021['two_digit'] = mesh_2021['TREENUMBER'].apply(lambda x:[i[:3] for i in x.split(';')])
mesh_2021.sample(5)

(29915, 3)
UI            0
NAME          0
TREENUMBER    0
dtype: int64


,UI,NAME,TREENUMBER,two_digit
17298,D000080871,Movable Books,V02.535,[V02]
19919,D046932,Nodularia,B03.280.550;B03.440.475.100.550;B03.585.625,"[B03, B03, B03]"
28692,D006884,Hydroxybutyrate Dehydrogenase,D08.811.682.047.820.350,[D08]
309,D049892,Mya,B01.050.500.644.080.535,[B01]
6893,D004271,"DNA, Fungal",D13.444.308.300,[D13]


time: 1.43 s (started: 2023-09-20 07:37:36 +00:00)


In [ ]:
def finder(terms):

  if str(terms)!='nan':
    mapped_list = []
    for i in terms:
      mapped_term = mesh_2021.loc[mesh_2021['NAME']==i]['two_digit'].tolist()
      if len(mapped_term)!=0 :
        mapped_list.append(mapped_term[0])

    return list(set([x for i in mapped_list for x in i ]))
  else:
    return terms

def find_two(df):

  df['three_digit_mesh'] = df['mesh_terms_mti'].apply(finder)
  return df

time: 1.55 ms (started: 2023-09-20 08:07:04 +00:00)


In [ ]:
kg_df = find_two(kg_df)

In [ ]:
kg_df.to_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0920_with_three.pickle')

time: 1.99 s (started: 2023-09-20 08:20:59 +00:00)


## lets go

In [ ]:
kg_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/kg_df_0920_with_three.pickle')
kg_df.rename(columns={'NCT_id':'NCT_ID'},inplace=True)
ade_result_matched_df = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ADE/ade_result_matched_df_0809.pickle')
inter_group_mesh_rel = pd.read_pickle(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/inter_group_mesh_rel_0829.pickle')

kg_df.rename(columns={'label':'significance'},inplace=True)
ade_result_matched_df.rename(columns={'serious/other':'serious_or_other'},inplace=True)

time: 11 s (started: 2023-09-30 01:16:42 +00:00)


In [ ]:
test_df = kg_df[kg_df['intervention_mesh'].str.len()==1]
test_df = test_df[test_df['intervention_type'].str.len()==1]
test_df['intervention_mesh'] = test_df['intervention_mesh'].apply(lambda x:x[0])
test_df['intervention_type'] = test_df['intervention_type'].apply(lambda x:x[0])

test_df.rename(columns={'intervention_group':'matched_group'},inplace=True)
test_df[['matched_group','intervention_mesh','intervention_type']].sample(10)

<ipython-input-20-9b31ac438a0c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['intervention_mesh'] = test_df['intervention_mesh'].apply(lambda x:x[0])
<ipython-input-20-9b31ac438a0c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['intervention_type'] = test_df['intervention_type'].apply(lambda x:x[0])
<ipython-input-20-9b31ac438a0c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

,matched_group,intervention_mesh,intervention_type
87659,Vibegron 75 mg,Tolterodine Tartrate,Drug
186984,BI 409306 Pooled,BI 409306,Drug
111157,Canagliflozin 300 mg,Canagliflozin,Drug
216288,LY900010 (Particle Size #2),Tadalafil,Drug
39169,LIK066 50 mg qd,Licogliflozin,Drug
7007,Risedronate,Risedronic Acid,Drug
180392,Treatment B (CHF 718 pMDI 400 µg TDD),Beclomethasone,Drug
221883,Group 1: QIV-HD,Vaccines,Biological
207598,Part 2: Placebo for MK-8521-T2DM,Liraglutide,Drug
64573,GW-1000-02,Nabiximols,Drug


time: 227 ms (started: 2023-09-30 01:41:24 +00:00)


In [ ]:
test_df.groupby(['intervention_group'])[['intervention_type','intervention_mesh']].value_counts().reset_index()

,intervention_group,intervention_type,intervention_mesh,0
0,0.017% P-321 Ophthalmic Solution,Drug,Pharmaceutical Solutions,6
1,0.03 mg SM04690,Drug,Lorecivivint,22
2,0.07 mg SM04690,Drug,Lorecivivint,22
3,0.1 Milligrams (mg) LY2189265,Drug,Dulaglutide,4
4,0.10% HL036 Ophthalmic Solution,Biological,Pharmaceutical Solutions,4
...,...,...,...,...
2827,iOWH032,Drug,Anti-Bacterial Agents,9
2828,iQuit Mindfully,Behavioral,Nicotine,1
2829,incobotulinumtoxinA (Xeomin),Drug,incobotulinumtoxinA,2
2830,rMenBOMV+ACWY_D Group,Biological,Vaccines,26


time: 49.8 ms (started: 2023-09-30 01:27:18 +00:00)


In [ ]:
ade_result_matched_df

,NCT_ID,arm_title,sub_title,category,serious_or_other,affected,at_risk,events,ratio,edge_type,matched_group,similarity
0,NCT01711294,19G Flex Needle,"Total, all-cause mortality",Total,serious,1.0,121.0,1.0,1/.121,has_ae,19G Flex Needle,NaN
1,NCT01711294,19G Flex Needle,"Total, serious adverse events",Total,serious,3.0,121.0,3.0,3/.121,has_ae,19G Flex Needle,NaN
2,NCT01711294,19G Flex Needle,Pancreatitis acute,Gastrointestinal disorders,serious,1.0,121.0,1.0,1/.121,has_ae,19G Flex Needle,NaN
3,NCT01711294,19G Flex Needle,Gastroenteritis,Infections and infestations,serious,1.0,121.0,1.0,1/.121,has_ae,19G Flex Needle,NaN
4,NCT01711294,19G Flex Needle,Endometrial cancer metastatic,"Neoplasms benign, malignant and unspecified (i...",serious,1.0,121.0,1.0,1/.121,has_ae,19G Flex Needle,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
804807,NCT02612428,Standard of Care (Control),Oesophageal Variceal Ligation,Surgical and medical procedures,serious,1.0,75.0,1.0,1/.75,has_ae,Standard of Care (Control),NaN
804808,NCT02612428,Standard of Care (Control),Hypotension,Vascular disorders,serious,3.0,75.0,3.0,3/.75,has_ae,Standard of Care (Control),NaN
804809,NCT02612428,Standard of Care (Control),Hypovolaemic Shock,Vascular disorders,serious,2.0,75.0,2.0,2/.75,has_ae,Standard of Care (Control),NaN
804810,NCT02612428,Standard of Care (Control),Orthostatic Hypotenstion,Vascular disorders,serious,1.0,75.0,1.0,1/.75,has_ae,Standard of Care (Control),NaN


time: 45.4 ms (started: 2023-09-30 01:17:12 +00:00)


In [ ]:
print(ade_result_matched_df.shape)

(803052, 12)
time: 641 µs (started: 2023-09-30 01:48:28 +00:00)


In [ ]:
merged_ade = pd.merge(ade_result_matched_df,test_df[['NCT_ID','matched_group','intervention_mesh','intervention_type']].drop_duplicates(),on=['NCT_ID','matched_group'],how='left')

time: 447 ms (started: 2023-09-30 01:58:01 +00:00)


In [ ]:
print(merged_ade.shape)

(803052, 14)
time: 510 µs (started: 2023-09-30 01:58:02 +00:00)


In [ ]:
merged_ade[merged_ade['intervention_mesh'].notna()]

,NCT_ID,arm_title,sub_title,category,serious_or_other,affected,at_risk,events,ratio,edge_type,matched_group,similarity,intervention_mesh,intervention_type
186,NCT01716754,QGE031 High Dose,"Total, serious adverse events",Total,serious,9.0,199.0,9.0,9/.199,has_ae,QGE031 240 mg q2w,0.890557,Omalizumab,Drug
187,NCT01716754,QGE031 High Dose,Coronary artery disease,Cardiac disorders,serious,1.0,199.0,1.0,1/.199,has_ae,QGE031 240 mg q2w,0.890557,Omalizumab,Drug
188,NCT01716754,QGE031 High Dose,Drug hypersensitivity,Immune system disorders,serious,1.0,199.0,1.0,1/.199,has_ae,QGE031 240 mg q2w,0.890557,Omalizumab,Drug
189,NCT01716754,QGE031 High Dose,Cellulitis,Infections and infestations,serious,1.0,199.0,1.0,1/.199,has_ae,QGE031 240 mg q2w,0.890557,Omalizumab,Drug
190,NCT01716754,QGE031 High Dose,Impetigo,Infections and infestations,serious,1.0,199.0,1.0,1/.199,has_ae,QGE031 240 mg q2w,0.890557,Omalizumab,Drug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802691,NCT02613572,Alpha Lipoic Acid (ALA) 1200mg (Phase II),Subarachnoid Haemorrhage,"Injury, poisoning and procedural complications",serious,1.0,26.0,1.0,1/.26,has_ae,Alpha Lipoic Acid (ALA) 1200mg,0.979870,Thioctic Acid,Drug
802692,NCT02613572,Alpha Lipoic Acid (ALA) 1200mg (Phase II),Cerebrovascular Accident,Nervous system disorders,serious,1.0,26.0,1.0,1/.26,has_ae,Alpha Lipoic Acid (ALA) 1200mg,0.979870,Thioctic Acid,Drug
802693,NCT02613572,Alpha Lipoic Acid (ALA) 1200mg (Phase II),Aspiration,"Respiratory, thoracic and mediastinal disorders",serious,1.0,26.0,1.0,1/.26,has_ae,Alpha Lipoic Acid (ALA) 1200mg,0.979870,Thioctic Acid,Drug
802694,NCT02613572,Alpha Lipoic Acid (ALA) 1200mg (Phase II),Pulmonary Embolism,"Respiratory, thoracic and mediastinal disorders",serious,1.0,26.0,1.0,1/.26,has_ae,Alpha Lipoic Acid (ALA) 1200mg,0.979870,Thioctic Acid,Drug


time: 268 ms (started: 2023-09-30 01:58:06 +00:00)


In [ ]:
merged_ade.groupby(['intervention_mesh','intervention_type'])[['affected']].sum().reset_index().sort_values(by=['affected'],ascending=False).head(10)

,intervention_mesh,intervention_type,affected
166,Darbepoetin alfa,Drug,16968.0
640,Vorapaxar,Drug,14575.0
163,Dapagliflozin,Drug,13267.0
174,Denosumab,Drug,11175.0
165,Darapladib,Drug,10860.0
214,Epoetin Alfa,Drug,10270.0
619,Vaccines,Biological,9967.0
592,Ticagrelor,Drug,9845.0
232,Evolocumab,Biological,9317.0
360,Liraglutide,Drug,9085.0


time: 119 ms (started: 2023-09-30 02:39:13 +00:00)


In [ ]:
merged_ade.groupby(['matched_group'])[['affected']].sum().reset_index().sort_values(by=['affected'],ascending=False).head(20)

,matched_group,affected
12788,Placebo,387640.0
4736,Control,39342.0
17484,Warfarin/Placebo Edoxaban,32282.0
8735,Intervention,30495.0
13114,Placebo Anacetrapib,28113.0
1879,Anacetrapib,27619.0
5290,Darbepoetin Alfa,24910.0
17481,Warfarin,24322.0
4915,Current BLS Training Program,24270.0
16383,Ticagrelor 60 mg,22793.0


time: 105 ms (started: 2023-09-30 02:27:40 +00:00)


In [ ]:
merged_ade.groupby('category')['affected'].sum().reset_index().sort_values(by=['affected'],ascending=False).head(11)

,category,affected
26,Total,1485514.0
10,Infections and infestations,294602.0
1,Cardiac disorders,286035.0
7,General disorders,195018.0
6,Gastrointestinal disorders,168440.0
22,"Respiratory, thoracic and mediastinal disorders",147184.0
15,"Neoplasms benign, malignant and unspecified (i...",132976.0
16,Nervous system disorders,129127.0
11,"Injury, poisoning and procedural complications",104784.0
27,Vascular disorders,93766.0


time: 80.5 ms (started: 2023-09-30 02:07:47 +00:00)


In [ ]:
merged_ade[merged_ade['category']!='Total']['affected'].sum()

2106063.0

time: 189 ms (started: 2023-09-30 02:21:56 +00:00)


In [ ]:
merged_ade[merged_ade['category']!='Total'].drop_duplicates(subset=['NCT_ID'])['at_risk'].sum()

3538554.0

time: 199 ms (started: 2023-09-30 02:24:16 +00:00)


In [ ]:
ade_result_matched_df.columns

Index(['NCT_ID', 'arm_title', 'sub_title', 'category', 'serious_or_other',
       'affected', 'at_risk', 'events', 'ratio', 'edge_type', 'matched_group',
       'similarity'],
      dtype='object')

time: 4.61 ms (started: 2023-09-28 03:02:06 +00:00)


In [ ]:
kg_df[['NCT_ID','intervention_group','intervention_value','intervention_mesh']].to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/efficacy_groups_0928.csv')

time: 540 ms (started: 2023-09-28 03:00:43 +00:00)


In [ ]:
kg_df[['NCT_ID','intervention_group','intervention_value','intervention_mesh']].sample(20)

,NCT_ID,intervention_group,intervention_value,intervention_mesh
102280,NCT00190749,Olanzapine,Drug,"[Olanzapine, Risperidone]"
98328,NCT00348140,RSG XR 8mg,Drug,[Rosiglitazone]
123858,NCT01035346,Ibuprofen Sodium,Drug,[Ibuprofen]
130438,NCT01227265,Preladenant 2 mg,Drug,[]
102881,NCT00420238,Etanercept,Drug,[Etanercept]
1853,NCT00212134,Aphakic Intraocular Lens,Device,[Dipivefrin]
93575,NCT00168818,Dabigatran 150mg,Drug,"[Enoxaparin, Dabigatran]"
133064,NCT01301456,Stage 2: PF-04856883 12 mg,Biological,[]
99528,NCT00385671,Gabapentin + Duloxetine,Drug,"[Pregabalin, Gabapentin, Duloxetine Hydrochlor..."
18712,NCT01380093,Morphine,Drug,"[Naltrexone, Morphine]"


time: 35.6 ms (started: 2023-09-28 03:05:51 +00:00)


In [ ]:
ade_result_matched_df[['NCT_ID','arm_title','matched_group']].to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/ade_groups_0928.csv')

time: 2.57 s (started: 2023-09-28 02:59:48 +00:00)


In [ ]:
infection_outcome_df = kg_df[kg_df['three_digit_mesh'].astype(str).str.contains('C01')|kg_df['outcome_title'].str.lower().str.contains('infect')]

infection_ade_df = ade_result_matched_df[ade_result_matched_df['sub_title'].str.lower().str.contains('infect')|ade_result_matched_df['category'].str.lower().str.contains('infect')]
infection_ade_df = infection_ade_df[infection_ade_df['matched_group'].isin(infection_outcome_df['intervention_group'].unique())]

infection_outcome_df = infection_outcome_df[infection_outcome_df['intervention_group'].isin(infection_ade_df['matched_group'].unique())]
pos_infection_outcome_df = infection_outcome_df[infection_outcome_df['significance']=='positive']
neg_infection_outcome_df = infection_outcome_df[infection_outcome_df['significance']=='negative']

infection_ade_df = infection_ade_df[infection_ade_df['matched_group'].isin(pos_infection_outcome_df['intervention_group'].unique())]

print(infection_outcome_df.shape,infection_outcome_df['NCT_ID'].nunique())
print(infection_ade_df.shape,infection_ade_df['NCT_ID'].nunique())
print(pos_infection_outcome_df.shape,pos_infection_outcome_df['NCT_ID'].nunique())
print(neg_infection_outcome_df.shape,neg_infection_outcome_df['NCT_ID'].nunique())

(1140, 49) 232
(14814, 12) 1533
(406, 49) 129
(734, 49) 177
time: 1.39 s (started: 2023-09-27 00:36:15 +00:00)


In [ ]:
print(pos_infection_outcome_df['intervention_group'].nunique())
unique_inters = set([item.lower() for sublist in pos_infection_outcome_df['intervention'].tolist() for item in sublist])
unique_inter_mesh = set([item.lower() for sublist in pos_infection_outcome_df['intervention_mesh'].tolist() for item in sublist])

unique_ade_groups = list(set(infection_ade_df['matched_group'].tolist()))
print(len(unique_ade_groups))
print(len(unique_inters))
print(len(unique_inter_mesh))

139
139
160
108
time: 4.14 ms (started: 2023-09-27 01:16:08 +00:00)


In [ ]:
print(unique_inter_mesh)

{'taurolidine', 'peginterferon alfa-2b', 'metronidazole', 'chloroquine', 'paclitaxel', 'acetaminophen', 'valganciclovir', 'lamivudine', 'efavirenz, emtricitabine, tenofovir disoproxil fumarate drug combination', 'terbinafine', 'loratadine', 'vincristine', 'adapalene', 'sulfadoxine', 'everolimus', 'antibodies', 'interferon alpha-2', 'ribavirin', 'trimethoprim, sulfamethoxazole drug combination', 'rho(d) immune globulin', 'immunoglobulins', 'methylprednisolone hemisuccinate', 'thrombin', 'peginterferon alfa-2a', 'darunavir', 'mycophenolic acid', 'alemtuzumab', 'diphenhydramine', 'nevirapine', 'tenofovir', 'cyclosporine', 'infliximab', 'cetuximab', 'cyclosporins', 'sirolimus', 'cobicistat', 'promethazine', 'gamma-globulins', 'trimethoprim', 'mometasone furoate', 'tofacitinib', 'emtricitabine, tenofovir disoproxil fumarate drug combination', 'thymoglobulin', 'maribavir', 'carboplatin', 'bamlanivimab', 'basiliximab', 'pharmaceutical solutions', 'gelatin sponge, absorbable', 'heparin', 'ataz

In [ ]:
print(set(unique_ade_groups))

{'Vaniprevir 24 Week Arm', 'Rabipur Group', '7vPnC After Infant Series Dose 3', 'MK-3415 + SOC', 'Nevirapine QD+BID', 'Vaccine', 'V710 60 µg', 'CD24Fc', 'Enhanced Treatment', 'Priorix-Tetra Group', 'CCI Vaccine', 'Group D: TDV Lyophilized', 'Control', 'HIV+/Cervarix Group', 'rhNGF 20 µg/ml', 'Telaprevir 24 Week+Peg-IFN-alfa-2a,RBV 48 Week', 'V419', 'Placebo + INFANRIX Hexa', 'QIVc (≥4 to <18 Years)', 'Baricitinib + SOC', 'ADV-TDF', 'Ziprasidone', 'Reltecimod (AB103) 0.5 mg/kg', 'Cohort 1-Engerix-B/Prevnar-Hiberix Group', 'Arm 1: Metronidazole Plus Miconazole', 'Neutrolin Arm', 'SB213503 Lot 1 + M-M-R Group', 'Azithromycin', 'DTG + RPV', 'Twinrix Group', 'IVV Vaccine', 'QIVc (≥18 Years)', '7vPnC After the Infant Series', 'ABC/ DTG/ 3TC', 'Immediate Introduction of Everolimus', 'Baloxavir Marboxil', 'PTX+ART', 'RotaTeq™', '4200mg Bamlanivimab', 'MMR Group', 'DTaP (Catch-up 7vPnC)', 'CAB LA+RPV LA (Q4W)', 'Placebo - Asia', 'Immediate ART', 'Telaprevir 12 Week+Peg-IFN-alfa-2a,RBV 48 Week',

In [ ]:
infection_ade_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/infection_ade_df.csv')
infection_outcome_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/infection_kg_df.csv')
pos_infection_outcome_df.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/pos_infection_outcome_df.csv')

inter_group_mesh_rel.to_csv(r'/content/drive/MyDrive/PKU/杜老师/data/ct.gov/22-12-25/KG/inter_group_mesh_rel_0829.csv')

time: 376 ms (started: 2023-09-27 00:41:03 +00:00)


In [ ]:
print(infection_ade_df.columns)
infection_outcome_df.columns

Index(['NCT_ID', 'arm_title', 'sub_title', 'category', 'serious_or_other',
       'affected', 'at_risk', 'events', 'ratio', 'edge_type', 'matched_group',
       'similarity'],
      dtype='object')


Index(['NCT_ID', 'study_type', 'completion_year', 'trial_phase',
       'enrollment_num', 'allocation', 'Countries', 'funder_name',
       'funder_type', 'condition_mesh', 'intervention_mesh', 'condition',
       'age', 'all_inters', 'intervention_type', 'intervention', 'comparator',
       'comparator_type', 'outcome_id', 'info', 'groups', 'outcome_type',
       'outcome_title', 'p_value', 'method', 'method_desc', 'ci_percent',
       'ci_n_sides', 'ci_lower_limit', 'ci_upper_limit', 'param_type',
       'param_value', 'split_p', 'p_split', 'significance',
       'ci_lower_limit_clean', 'ci_upper_limit_clean', 'mesh_terms_mti',
       'mapped_mesh_terms', 'tree_level_0', 'intervention_dict',
       'comparator_dict', 'intervention_group', 'intervention_value',
       'intervention_similarity', 'comparator_group', 'comparator_value',
       'comparator_similarity', 'three_digit_mesh'],
      dtype='object')

time: 6.42 ms (started: 2023-09-27 00:28:10 +00:00)


In [ ]:
pd.merge(infection_ade_df[['arm_title','sub_title']].rename(columns={'arm_title':'intervention_group'}),infection_outcome_df[['intervention_group','outcome_title','significance']],on='intervention_group',how='inner').sample(20)

,intervention_group,sub_title,outcome_title,significance
67653,Cyclosporine,Necrotising fasciitis,Kaplan-Meier Analysis of Percentage of Partici...,negative
76600,Telaprevir 24 Week+Peg-IFN-alfa-2a 24 Week,Gastroenteritis,Percentage of Subjects With Undetectable Plasm...,positive
35989,Placebo,Infected cyst,The Change From Baseline in Rhinitis Symptom S...,negative
17985,Placebo,INFECTED SKIN ULCER,The Change From Baseline in Rhinitis Medicatio...,negative
53343,Placebo,Abscess,The Change From Baseline in Rhinitis Medicatio...,negative
41326,Placebo,Sepsis,Mean of the Sum of 5 Single Local Common Cold ...,negative
35267,Placebo,Soft tissue infection,Number of Participants With Active COVID-19 Di...,negative
30183,Placebo,Wound infection staphylococcal,The Change From Baseline in Rhinitis Medicatio...,negative
21870,Placebo,Pneumonia viral,Incidence of Confirmed Herpes Zoster (HZ) Case...,positive
63411,Control,Pulmonary sepsis,Sepsis,negative


time: 32.8 ms (started: 2023-09-27 00:29:36 +00:00)
